In [69]:
"""Collect tracks list."""
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from playlist_selection.tracks.constants import Genres

/home/slava/.cache/pypoetry/virtualenvs/project-ha1ywNtr-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
from playlist_selection.parsing import SpotifyParser

client_id = ""
client_secret = ""

parser = SpotifyParser(
    client_id=client_id,
    client_secret=client_secret,
)

In [76]:
np.random.seed(42)

OFFSETS = [0, 50, 100, 150, 200]
DATES = pd.date_range("1990-01-01", "2025-01-01", freq="5Y")

track_collection = []
for genre in tqdm(Genres):
    genre_query = f'genre:"{genre.value}"'
    
    for start_date in DATES:
        start_date = start_date.date()
        end_date = start_date + pd.to_timedelta(365 * 5, unit="days")
        query = f'{genre_query} year:"{start_date.year}"-"{end_date.year}"'
        
        for i, offset in enumerate(reversed(OFFSETS), start=1): # quavo   
            results = parser.sp.search(
                q=query, 
                type='track',
                limit=50,
                offset=offset,
            )
            
            sample_size = min(3 * i, len(results["tracks"]["items"]))
            if sample_size == 0:
                continue

            sample = np.random.choice(results["tracks"]["items"], sample_size, replace=False)
            
            for track in sample:
                artist = track["artists"][0]["name"]
                name = track["name"]
                track_collection.append((genre.value, name, artist))

100%|██████████| 126/126 [22:38<00:00, 10.78s/it]


In [77]:
with open("../data/tracks.txt", "w") as fout:
    for line in track_collection:
        line = "\t".join(line)
        print(line, file=fout)